<a href="https://colab.research.google.com/github/niyatisrivastava/MAJOR_PROJECT/blob/main/Sentiment_Analysis_Latest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install streamlit
!pip install pyngrok==4.1.1
from pyngrok import ngrok

     |████████████████████████████████| 7.0MB 8.9MB/s 
     |████████████████████████████████| 4.6MB 42.1MB/s 
     |████████████████████████████████| 112kB 53.5MB/s 
     |████████████████████████████████| 163kB 42.1MB/s 
     |████████████████████████████████| 81kB 7.7MB/s 
     |████████████████████████████████| 122kB 51.8MB/s 
     |████████████████████████████████| 71kB 6.7MB/s 
  Created wheel for blinker: filename=blinker-1.4-cp37-none-any.whl size=13448 sha256=1b5e4c9d9805d23f9954bb32f907a845b38fe703030bf56017d220c786d6a067
  Stored in directory: /root/.cache/pip/wheels/92/a0/00/8690a57883956a301d91cf4ec999cc0b258b01e3f548f86e89
Successfully built blinker
ERROR: google-colab 1.0.0 has requirement ipykernel~=4.10, but you'll have ipykernel 5.5.0 which is incompatible.
  Found existing installation: ipykernel 4.10.1
    Uninstalling ipykernel-4.10.1:
      Successfully uninstalled ipykernel-4.10.1


  Created wheel for pyngrok: filename=pyngrok-4.1.1-cp37-none-any.whl size=15971 sha256=e41ce26ba297eb9c88833dc69ba316d5055826bcca65bc9907ce1efd3a2ea02d
  Stored in directory: /root/.cache/pip/wheels/97/71/0d/1695f7c8815c0beb3b5d9b35d6eec9243c87e6070fbe3977fa
Successfully built pyngrok


In [ ]:
!pip install contractions

     |████████████████████████████████| 266kB 7.8MB/s 
     |████████████████████████████████| 327kB 15.4MB/s 
  Created wheel for pyahocorasick: filename=pyahocorasick-1.4.1-cp37-cp37m-linux_x86_64.whl size=85265 sha256=54cdbbfd20af11719db5bd4ec1717d8967906bc8033c1224b737d902e6af75bc
  Stored in directory: /root/.cache/pip/wheels/e4/ab/f7/cb39270df8f6126f3dd4c33d302357167086db460968cfc80c
Successfully built pyahocorasick


In [ ]:
!pip install vaderSentiment

     |████████████████████████████████| 133kB 9.1MB/s 


In [ ]:
%%writefile sentiment.py

import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import numpy as np
import os

url = 'https://inshorts.com/en/read/business'
news_data = []
news_category = url.split('/')
#news_category

data = requests.get(url)
soup = bs(data.content)
#print(soup)

urls = ['https://inshorts.com/en/read/business', 'https://inshorts.com/en/read/politics', 'https://inshorts.com/en/read/startup', 'https://inshorts.com/en/read/entertainment']
def build_dataset(urls):
  news_data = []
  for url in urls:
    news_category = url.split('/')
    data = requests.get(url)
    soup = bs(data.content)
     
    news_articles = [{'news_headline': headline.find('span', attrs = {"itemprop":"headline"}).string,
                      'news_article': article.find('div', attrs = {"itemprop":"articleBody"}).string,
                      'news_category': news_category}
                     
                     for headline,article in zip(soup.find_all('div', class_=["news-card-title news-right-box"]),
                                                soup.find_all('div', class_=["news-card-content news-right-box"]))]
    news_articles = news_articles[0:20]
    news_data.extend(news_articles)
 
  df = pd.DataFrame(news_data)
  df = df[['news_headline', 'news_article','news_category']]
  return df

df = build_dataset(urls)
df.head()

df.to_csv('news.csv', index=False)

import pandas as pd
df = pd.read_csv('news.csv')

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
 
stop_words = stopwords.words('english')

import re
def remove_sp(text):
  pattern  = r'[^A-Za-z0-9]'
  text = re.sub(pattern, ' ', text)
  return text
 
def html_tag(text):
  soup = bs(text, "html.parser")
  new_text = soup.get_text()
  return new_text
 
import contractions
def con(text):
  expand = contractions.fix(text)
  return expand
 
from nltk.tokenize.toktok import ToktokTokenizer
tokenizer = ToktokTokenizer()
 
def remove_stopwords(text):
  tokens = tokenizer.tokenize(text)
  tokens = [token.strip() for token in tokens]
  filtered_tokens = [token for token in tokens if token not in stop_words]
  filtered_text = ' '.join(filtered_tokens)
  return filtered_tokens

df.news_headline = df.news_headline.apply(lambda x:x.lower())
df.news_article = df.news_article.apply(lambda x:x.lower())

df.news_headline = df.news_headline.apply(html_tag)
df.news_article = df.news_article.apply(html_tag)

df.news_headline = df.news_headline.apply(con)
df.news_article = df.news_article.apply(con)

df.news_headline = df.news_headline.apply(remove_sp)
df.news_article = df.news_article.apply(remove_sp)

df.news_headline = df.news_headline.apply(remove_stopwords)
df.news_article = df.news_article.apply(remove_stopwords)

String = ""
 
for i in range (0,len(df.index)) :
  for j in range (0,len(df['news_article'][i])) : 
    String = String + df['news_article'][i][j]+" "
  df['news_article'][i]=String
  String = ""

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
vs = SentimentIntensityAnalyzer()

df['compound'] = df['news_article'].apply(lambda news_article: vs.polarity_scores(news_article)['compound'])

Store = []
for i in range (0,len(df.index)) :
  if df['compound'][i]>0 : Store.append('Positive')
  elif df['compound'][i]<0 : Store.append('Negative')
  elif df['compound'][i]==0 : Store.append('Neutral')  
  
import streamlit as st 
st.title('Sentiment Analysis')

df["Sentiment"] = Store

df
select = st.text_input('Enter text')
s=vs.polarity_scores(select)
result = ''
flag=0
#st.title(result)
if s['compound'] > 0 : 
  result='The news contains a positive sentiment'
  flag=flag+1
elif s['compound'] < 0 :
  result='The news contains a negetive sentiment'
  flag=flag+1
elif s['compound'] == 0 :
  result='The news contains a neutral sentiment'
  flag=flag+1
#st.title
#if flag>1:
st.title(result)



Overwriting sentiment.py


In [ ]:
!nohup streamlit run sentiment.py &
url = ngrok.connect(port='8501')
url

nohup: appending output to 'nohup.out'


'http://f2dbe92ff1b0.ngrok.io'